In [1]:
import sys
import warnings
import panel as pn
import numpy as np

# DO NOT remove this line below, otherwise the app won't be found
sys.path.append('..')
warnings.filterwarnings('ignore')

In [2]:
import sim2bids
from sim2bids.sim2bids import MainArea
from sim2bids.generate import models 

pn.extension('tabulator', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width', notifications=True)

NotificationArea(sizing_mode='stretch_width')

In [4]:
path = '../output'
sys.path.append(path)

# specify requirements 
sim2bids.app.app.SoftwareVersion = 1.5
sim2bids.app.app.SoftwareRepository = 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10'
sim2bids.app.app.SoftwareName = 'TVB'

# specify model used if the code is not in Python
sim2bids.app.app.MODEL_NAME = 'reduced_wong_wang'   # options: reduced_wong_wang, hindmarsh_rose, generic2doscillator
sim2bids.app.app.MODEL_PARAMS = dict(G=np.arange(0.01, 3, 0.015))

app = MainArea()
pn.serve(app.view())

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:56139


INFO:tornado.access:200 GET / (127.0.0.1) 188.02ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/jsoneditor.min.css (127.0.0.1) 1.96ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/ajax/libs/font-awesome/5.15.1/css/all.min.css (127.0.0.1) 2.02ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/datatabulator/tabulator-tables@4.9.3/dist/css/tabulator_simple.min.css (127.0.0.1) 5.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/npm/notyf@3/notyf.min.css (127.0.0.1) 9.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/alerts.css (127.0.0.1) 3.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/card.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/dataframe.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/json.css (127.0.0.1) 7.00ms
INFO:tornado.access:200 GET /static/extensions/pa

FILES: ['CHANGES', 'dataset_description.json', 'participants.tsv', 'README', 'sub-CON01', 'sub-CON02']
SID @82: sub-01
SID @92: sub-01
SID @126 sub-01



C:\Users\dinar\Desktop\TVB\sub-CON01\ses-preop\ts.txt
SID @82: sub-02
SID @92: sub-02
SID @126 sub-02



C:\Users\dinar\Desktop\TVB\sub-CON02\ses-preop\ts.txt
SID @97: sub-02
SID @126 sub-02



C:\Users\dinar\Desktop\TVB\sub-CON02\ses-postop\ts.txt
OrderedDict([('sub-01', {'ses-preop': OrderedDict([('areas', {'name': 'areas', 'fname': 'areas.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\TVB\\sub-CON01\\ses-preop\\areas.txt', 'ext': 'txt'}), ('centres', {'name': 'centres', 'fname': 'centres.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\TVB\\sub-CON01\\ses-preop\\centres.txt', 'ext': 'txt'}), ('cortical', {'name': 'cortical', 'fname': 'cortical.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\TVB\\sub-CON01

In [9]:
import os
files = os.listdir('output')

missed = lambda x: x.split('.')[0] in ['CHANGES', 'README', 'participants', 'dataset_description']
missing = [x for x in files if not missed(x) and not os.path.isdir(os.path.join('output', x))]
missing

[]

In [9]:
import sim2bids
from sim2bids.app import app

def accepted(name, return_accepted=False):
    for accept in app.ACCEPTED:
        if accept in name:
            if accept == 'ts' or accept == 'times':
                if accept == 'ts':
                    if len(name) != 2 or '_ts' not in name:
                        return False
                split = name.split('_')
                if len(split) == 2:
                    if return_accepted:
                        return accept
                    return True, split[-2] + '_' + split[-1]
                if len(split) >= 3:
                    if return_accepted:
                        return accept
                    return True, split[-3] + '_' + split[-2] + '_' + split[-1]

                if return_accepted:
                    return accept

                return True, accept

            if accept == 'emp' and 'emp_fc' in name:
                if return_accepted:
                    return accept
                return True, 'emp_fc'
            
            if accept == 'fc' and len(name.split('_')) > 1:
                if return_accepted:
                    return accept
                return False
            
            if accept != 'ts':
                if return_accepted:
                    return accept
                return True, accept
    return False

In [10]:
accepted('fc_mi')

(True, 'fc')

In [5]:
import numpy as np
import pandas as pd 


open('C:/Users/dinar/Desktop/TVB/CHANGES').readlines()

['Please add additional changes with dates and version numbers of dataset here.\n',
 '\n',
 '1.0.0 2019-10-06\n',
 '\n',
 '  - initial release\n']

In [9]:
np.load('C:/Users/dinar/Desktop/TVB/CHANGES')

UnpicklingError: Failed to interpret file 'C:/Users/dinar/Desktop/TVB/CHANGES' as a pickle

In [5]:
import os
import json
import shutil
import numpy as np
import lems.api as lems
from sim2bids.templates import model_params
from sim2bids.generate import structure
from sim2bids.app import app
from sim2bids.convert import convert
from sim2bids.templates import templates


class Model:
    def __init__(self, model_name, rhythms=None, **kwargs):
        self.model_name = model_name
        self.possible_params = kwargs
        self.params = self.get_params()

        # pylems model parameters
        self.id = 1

        # check output structure and create folders if necessary
        structure.check_folders(app.OUTPUT)

        self.set_params()

    def get_params(self):
        if self.model_name == 'reduced_wong_wang':
            return model_params.reduced_wong_wang
        elif self.model_name == 'hindmarsh_rose':
            return model_params.hindmarsh_rose
        elif self.model_name == 'generic2doscillator':
            return model_params.g2dos

    def set_params(self):
        for k, v in self.possible_params.items():
            if not isinstance(v, float) and self.params.get(k, None) is not None:
                self.save_params(k, v)

    def save_params(self, k, v):
        # save equations

        # ==================================================================
        # NOTE: IF YOU'RE USING A LOCAL VERSION FORKED/DOWNLOADED FROM 
        # GITHUB, UNCOMMENT THE LINE BELOW AND COMMENT OUT THE FILE LOCATION
        # THIS WILL ALLOW YOU READING THE DEFAULT XML MODELS 
        # ==================================================================

        # COMMENT OUT THESE LINES OF CODE IF YOU'RE USING A LOCAL VERSION OF THE APP
        xml = os.path.join(f'../sim2bids/models/{self.model_name}.xml')

        # UNCOMMENT THESE LINES OF CODE IF YOU'RE USING LOCAL VERSION OF THE APP
        # here = os.path.dirname(os.path.abspath(__file__))
        # xml = os.path.join(here, 'models', self.model_name + '.xml')

        # copy the default equations xml file
        path = os.path.join(app.OUTPUT, 'eq', f'desc-{app.DESC}_eq.xml')
        shutil.copy(xml, path)
        convert.to_json(path.replace('xml', 'json'), shape=None,
                        desc=templates.file_desc['eq'].format(self.model_name.upper()), key='eq')

        # save params
        # TODO: ADD DIFFERENT RHYTHMS TRAVERSAL

        # iterate over values
        for value in v:
            path = os.path.join(app.OUTPUT, 'param', f'desc-{app.DESC}-{k}{str(format(value, ".4f"))}.xml')
            save_json(path, self.get_model(dict(k=float(value))), use_json=False)
            convert.to_json(path.replace('xml', 'json'), shape=None,
                            desc=templates.file_desc['param'].format(self.model_name.upper()), key='param')

    def get_model(self, value):
        # instantiate a LEMS model 
        model = lems.Model()

        # supply values
        model.add(lems.Component(id_=self.id, type_=self.model_name, **value))

        # increase the id value
        self.id += 1

        return model


def save_json(path, model=None, use_json=True):
    if model:
        model.export_to_file(path)
    
    if use_json:
        with open(path, 'w') as f:
            json.dump(model, f)



In [6]:
app.SoftwareVersion = 1.5
app.SoftwareRepository = 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10'
app.SoftwareName = 'TVB'

Model('reduced_wong_wang', G=np.arange(0.01, 3, 0.015))

PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'SourceCode': None, 'SoftwareVersion': 1.5, 'SoftwareRepository': 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10', 'SourceCodeVersion': 1.5, 'SoftwareName': 'TVB', 'Network': None}
PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'SourceCode': None, 'SoftwareVersion': 1.5, 'SoftwareRepository': 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10', 'SourceCodeVersion': 1.5, 'SoftwareName': 'TVB', 'Network': None}
PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'SourceCode': None, 'SoftwareVersion': 1.5, 'SoftwareRepository': 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10', 'SourceCodeVersion': 1.5, 'SoftwareName': 'TVB', 'Network': None}
PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'Sour

PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'SourceCode': None, 'SoftwareVersion': 1.5, 'SoftwareRepository': 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10', 'SourceCodeVersion': 1.5, 'SoftwareName': 'TVB', 'Network': None}
PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'SourceCode': None, 'SoftwareVersion': 1.5, 'SoftwareRepository': 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10', 'SourceCodeVersion': 1.5, 'SoftwareName': 'TVB', 'Network': None}
PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'SourceCode': None, 'SoftwareVersion': 1.5, 'SoftwareRepository': 'https://github.com/the-virtual-brain/tvb-root/releases/tag/1.5.10', 'SourceCodeVersion': 1.5, 'SoftwareName': 'TVB', 'Network': None}
PARAMS: {'ModelEq': '../eq/desc-default_eq.xml', 'ModelParam': '../param/desc-default_param.xml', 'Sour

In [22]:
app.SoftwareName

'TVB'

In [12]:
import numpy as np

len(np.arange(0.01, 3, 0.015))

200

In [17]:
os.listdir()

['.ipynb_checkpoints', 'output', 'start_app.ipynb']

In [4]:
paths = ['C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\con01t1_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\con01t1_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\HRF.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\para.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\roi_id_table.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\scthran.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\con02t2_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\con02t2_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\hrf.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\para.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\ROI.dat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\roi_id_table.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-postop\\scthran.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\con02t1_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\con02t1_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\hrf.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\para.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\roi_id_table.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON02\\ses-preop\\scthran.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\con03t2_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\con03t2_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\hrf.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\para.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\roi_id_table.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-postop\\scthran.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\con03t1_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\con03t1_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\hrf.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\para.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\roi_id_table.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON03\\ses-preop\\scthran.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\con04t2_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\con04t2_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\hrf.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\hrf.txt_map.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\para.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\ROIts.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\roi_id_table.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-postop\\scthran.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\con04t1_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\con04t1_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\hrf.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\hrf.txt_map.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\para.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\ROIts.dat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\ROIts.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\roi_id_table.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\SCthrAn.mat', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON04\\ses-preop\\scthran.txt']
    
    

In [17]:
import os

def get_files(paths, file_name, constraint, search1dir=False):
    files, dir_name = [], os.path.dirname(paths[0])

    if search1dir:
        paths = [os.path.join(dir_name, f) for f in os.listdir(dir_name)]

    for path in paths:
        if file_name in path or constraint in path:
            files.append(path)

    return files


# def iterate_folder(path):
    

In [18]:
get_files(paths, 'scthran', 'weights', search1dir=True)

['C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\areas.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\average_orientations.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\centres.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\con01t1_roits.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\con01t1_roits_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\cortical.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\fc_cc.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\fc_cc_dk68.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\fc_mi.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\hemisphere.txt', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\HRF.csv', 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\HR

['C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\scthran.txt',
 'C:\\Users\\dinar\\Desktop\\brain_tumor_copy\\sub-CON01\\ses-preop\\weights.txt']

In [ ]:
import os

import pandas as pd
import panel as pn
from scipy.io import loadmat
import mat73
import scipy

import sim2bids.generate.subjects as subj
from sim2bids.app import app

RENAMED = []


def filter(contents, files=None):
    if files is None:
        files = []

        for content in contents:
            if not subj.accepted(content):
                match = subj.find_matches([content])
                if len(match) > 0:
                    files.append(content.replace(match[0], '').replace('_', ''))
                else:
                    files.append(content)

        return list(set(files))

    # else get paths to files
    paths = []

    for content in contents:
        for file in files:
            if file in content:
                paths.append(content)

    return paths


def validate(unique_files, paths):
#     to_rename = []

    for idx, file in enumerate(unique_files):
        if type(file) == pn.widgets.select.Select:
            name, value = file.name.replace('Specify ', ''), file.value
            ext = name.split('.')[-1]

            if subj.accepted(name):
                print(name)
                to_rename.append(value)

                if verify_weights(name):
                    rename_files(name, value, paths)
            elif value == 'weights & nodes':
                result = verify_weights_nodes(name, paths)

                if isinstance(result, bool):
                    pass
                elif isinstance(result, list):
                    extract_files(name, result[1], result[-1], paths)

            # if the selection's value is "skip", remove the file from input folder
            elif value == 'skip':
                remove_files(name, paths)
            elif value == 'map':
                if ext in ['csv', 'dat', 'txt']:
                    rename_files(name, 'map', paths)
            elif value == 'ts':
                if ext in ['csv', 'dat', 'txt']:
                    rename_files(name, 'ts', paths)
            elif value == 'fc':
                rename_files(name, 'fc', paths)

    # let users know that files were successfully renamed
#     if len(to_rename) == len(RENAMED):
#         pn.state.notifications.success('Renamed all files!')


def verify_weights(name):
    ext = get_ext(name)

    if ext not in ['txt', 'csv']:
        pn.state.notifications.error('Weights should be in CSV, TXT format. '
                                     'Please double-check your selection')
        return False

    return True


def verify_weights_nodes(name, all_files):
    ext = get_ext(name)

    if ext == 'mat':
        mat, cols = open_mat(get_file(all_files, name))

        if 'sc' not in [x.lower() for x in cols]:
            pn.state.notifications.error('Weights are not found in MATLAB file. Please double-check your selection')
            return False
        else:
            if 'ids' in cols:
                return [True, mat, ['sc', 'ids']]


def extract_files(ext, mat, cols, paths):
    # check if the structure is multi-file in one folder
    matches = subj.find_matches([os.path.basename(x) for x in paths])
    base = os.path.basename(paths[0])
    og_path = paths[0].replace(base, '')

    for match in matches:
        for path in paths:
            if os.path.basename(path).startswith(match) and path.endswith(ext):
                mat[cols[0]].tofile(os.path.join(og_path, f'{match}_weights.txt'), sep=' ')
                pd.DataFrame(get_nodes(mat[cols[-1]])).to_csv(os.path.join(og_path, f'{match}_nodes.txt'),
                                                              header=None, index=None)
                # delete file
                os.remove(path)


def get_nodes(arr: list) -> list:
    all_nodes = []

    for node in arr:
        split = [x for x in node[0].split(' ') if x.startswith('ctx')]
        if len(split) > 0:
            all_nodes.append(split[0].replace('ctx', '').replace('-', '_').strip('-_/?.!,'))

    return all_nodes


def rename_files(name, new_ext, paths):
    global RENAMED
    new_ext = new_ext.replace('.', '').lower()

    for file in paths:
        if file.endswith(name):
            if file.endswith('txt') or file.endswith('csv') or file.endswith('dat'):
                p = os.path.dirname(file)
                if new_ext == 'weights' and 'weights.txt' in os.listdir(p):
                    os.rename(file, os.path.join(p, f'weights_{name}'))
                    os.rename(os.path.join(p, 'weights.txt'), os.path.join(p, 'weights_SCnotthrAn.txt'))
                else:
                    try:
                        os.rename(file, file.replace(name, f'{name.replace(".txt", "")}_{new_ext}' + '.txt'))
                    except FileExistsError:
                        pn.state.notifications.error(f'File {new_ext} already exists!')
                    else:
                        RENAMED.append(file)
            elif file.endswith('mat'):
                pass


def get_file(files, end):
    for file in files:
        if file.endswith(end):
            return file


def get_ext(file):
    return os.path.basename(file).split('.')[-1]


def open_mat(file):
    try:
        mat = loadmat(file, squeeze_me=True)
    except NotImplementedError:
        mat = mat73.loadmat(file)
    except scipy.io.matlab._miobase.MatReadError:
        return

    return mat, find_mat_array(mat)


def find_mat_array(mat):
    data = []

    for k, v in mat.items():
        if type(v) not in [bytes, str, list]:
            data.append(k)

    return data


def remove_files(name, all_files):
    for file in all_files:
        if file.endswith(name):
            os.remove(file)

def get_extensions(files, ids=None):
    to_rename = []

    if ids is not None:
        files = [x for x in list(set([remove_id(x, ids) for x in files])) if x is not None]

    for file in files:
        found = False
        for acc in app.ACCEPTED:
            if file.lower().startswith(acc.lower()):
                found = True

        if not found:
            to_rename.append(file)

    return list(set(to_rename))


def remove_id(file, ids):
    for id_ in ids:
        if file.startswith(id_):
            return file.replace(id_, '').strip(',.\\/_;!?-')

In [5]:
from scipy.io import loadmat

loadmat('C:/Users/dinar/Desktop/brain_tumor_copy/sub-CON01/ses-preop/FC.mat').keys()

dict_keys(['__header__', '__version__', '__globals__', 'CON01T1_ROIts', 'CON01T1_ROIts_DK68', 'FC_cc_DK68', 'FC_cc', 'FC_mi', 'ROI_ID_table'])

INFO:bokeh.server.views.ws:WebSocket connection closed: code=1001, reason=None


In [10]:
import pandas as pd

path1 = 'C:/Users/dinar/Desktop/brain_tumor/sub-CON04/ses-preop/HRF.csv'
path2 = 'C:/Users/dinar/Desktop/brain_tumor_copy/sub-CON04/ses-preop/HRF.csv'

pd.read_csv(path1, header=None, sep=',')

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,-6.424144e-17,-1.871891e-17,-2.145912e-17,-9.136016e-17,-2.546784e-17,-2.888010e-17,-4.636552e-17,-2.679997e-17,-7.286268e-17,-6.430698e-17,...,-3.690437e-17,-2.356070e-17,-3.398875e-17,-5.926875e-17,-2.243488e-17,-3.988098e-17,-4.096134e-17,-3.346029e-17,-8.568627e-17,-3.104507e-17
1,1.582964e-03,-2.272392e-04,-1.369279e-04,1.996538e-03,6.593850e-04,7.124876e-04,8.284793e-04,5.792457e-04,1.788632e-03,1.045618e-03,...,8.616980e-04,-3.639361e-04,7.649824e-04,1.556024e-03,2.353038e-04,7.910210e-04,9.815970e-04,8.328948e-04,2.508034e-03,5.877908e-04
2,3.178766e-03,-4.563387e-04,-2.749713e-04,4.009269e-03,1.324119e-03,1.430753e-03,1.663671e-03,1.163189e-03,3.591770e-03,2.099699e-03,...,1.730381e-03,-7.308339e-04,1.536166e-03,3.124667e-03,4.725104e-04,1.588453e-03,1.971154e-03,1.672548e-03,5.036414e-03,1.180345e-03
3,4.787459e-03,-6.873035e-04,-4.141321e-04,6.038260e-03,1.994224e-03,2.154821e-03,2.505604e-03,1.751851e-03,5.409475e-03,3.162281e-03,...,2.606079e-03,-1.100702e-03,2.313578e-03,4.705981e-03,7.116293e-04,2.392323e-03,2.968705e-03,2.518988e-03,7.585222e-03,1.777683e-03
4,6.409096e-03,-9.201382e-04,-5.544120e-04,8.083580e-03,2.669723e-03,2.884714e-03,3.354307e-03,2.345250e-03,7.241808e-03,4.233402e-03,...,3.488820e-03,-1.473548e-03,3.097242e-03,6.300018e-03,9.526698e-04,3.202660e-03,3.974283e-03,3.372243e-03,1.015454e-02,2.379825e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24495,-1.539252e-03,-2.979478e-02,2.327586e-02,1.672383e-02,3.635319e-03,-2.185167e-03,-9.279625e-03,6.123568e-03,2.632073e-03,-3.830704e-02,...,-1.184735e-02,2.010360e-02,-7.375084e-03,-1.082100e-02,-2.157797e-03,-6.075505e-03,-6.721925e-04,1.547007e-02,-6.748408e-03,-4.976266e-03
24496,-1.229235e-03,-2.379399e-02,1.858802e-02,1.335559e-02,2.903153e-03,-1.745064e-03,-7.410666e-03,4.890259e-03,2.101968e-03,-3.059184e-02,...,-9.461245e-03,1.605466e-02,-5.889710e-03,-8.641606e-03,-1.723204e-03,-4.851870e-03,-5.368070e-04,1.235434e-02,-5.389248e-03,-3.974023e-03
24497,-9.203011e-04,-1.781409e-02,1.391649e-02,9.999078e-03,2.173536e-03,-1.306494e-03,-5.548218e-03,3.661244e-03,1.573707e-03,-2.290351e-02,...,-7.083448e-03,1.201982e-02,-4.409510e-03,-6.469800e-03,-1.290127e-03,-3.632498e-03,-4.018945e-04,9.249454e-03,-4.034823e-03,-2.975271e-03
24498,-6.124497e-04,-1.185512e-02,9.261306e-03,6.654302e-03,1.446469e-03,-8.694589e-04,-3.692287e-03,2.436527e-03,1.047291e-03,-1.524208e-02,...,-4.713970e-03,7.999089e-03,-2.934488e-03,-4.305592e-03,-8.585660e-04,-2.417393e-03,-2.674555e-04,6.155431e-03,-2.685137e-03,-1.980015e-03


In [19]:
a = loadmat('C:/Users/dinar/Desktop/brain_tumor/sub-CON04/ses-preop/SCthrAn.mat')['SCthrAn']
b = pd.read_csv('C:/Users/dinar/Desktop/brain_tumor_copy/sub-CON04/ses-preop/weights.txt', header=None, sep='\s').values
b

array([[0.00000000e+00, 0.00000000e+00, 1.52000000e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.20000000e-04, ...,
        0.00000000e+00, 0.00000000e+00, 6.66666667e-05],
       [1.52000000e-03, 7.20000000e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.09333333e-03],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 5.17200000e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.45333333e-03],
       [0.00000000e+00, 6.66666667e-05, 1.09333333e-03, ...,
        5.17200000e-02, 1.45333333e-03, 0.00000000e+00]])

In [23]:
import numpy as np

np.array_equal(a, b)

False

In [5]:
# import local packages
import sim2bids.utils
from sim2bids.generate import structure, subjects
from sim2bids.preprocess import preprocess as prep
from sim2bids.preprocess import prepare_inputs as prepare
from sim2bids.convert import convert, mat
from sim2bids.templates import templates as temp
from sim2bids.app import utils

import json

def check_json():
    for file in utils.get_content(OUTPUT, os.listdir(OUTPUT)):
        if file.endswith('json'):
            f = json.load(open(file))
            if convert.COORDS is not None and 'CoordsRows' in f.keys() and f['CoordsRows'] == '':
                if 'coord' in file:
                    coords = [convert.COORDS[0].replace('json', 'tsv'), convert.COORDS[1].replace('json', 'tsv')]
                    f['CoordsRows'] = coords
                    f['CoordsColumns'] = coords
                else:
                    f['CoordsRows'] = convert.COORDS
                    f['CoordsColumns'] = convert.COORDS

            with open(file, 'w') as f2:
                json.dump(f, f2)


In [29]:
import os 
from sim2bids.app import app
from sim2bids.generate import subjects

for root,dirs,files in os.walk(app.OUTPUT):
    # get level number from the root folder
    level = len(root.split('\\'))
    
    for file in files:
        if file.endswith('json'):
            path = os.path.join(root, file)
            json_file = json.load(open(path))
            
            if convert.IGNORE_CENTRE:
                lvl = '../' * (level - 1) 
                coords = [f'{lvl}coord/desc-{app.DESC}_labels.json', f'{lvl}coord/desc-{app.DESC}_nodes.json']
            else:
                if 'sub-' in path:
                    if 'coord' in root:
                        coords = None
                    else:
                        sub = re.findall(r'sub-[0-9]+', path)[0]
                        coords = [f'../coord/{sub}_desc-{app.DESC}_labels.json', f'../coord/{sub}_desc-{app.DESC}_nodes.json']
            
            # check if CoordRows and CoordColumns exist
            if 'CoordsRows' in json_file.keys():
                # remove CoordRows and CoordColumns from nodes/labels
                if 'nodes' in file or 'labels' in file or coords is None:
                    del json_file['CoordsRows']
                    del json_file['CoordsColumns']
                else:
                    json_file['CoordsRows'] = coords
                    json_file['CoordsColumns'] = coords
                
                # save changes
                with open(path, 'w') as f:
                    json.dump(json_file, f)


In [21]:
import re

re.findall(r'sub-[0-9]+', 'output\\sub-03\\ses-preop\\net\\sub-03_desc-default_weights.json')[0]

'sub-03'

In [9]:
import os
import zipfile
import sim2bids.generate.subjects as subj


def extract_zip(path):
    # get folder name
    basename = subj.get_filename(path)

    # get root directory
    parent = path.replace(basename, '')

    # files to extract
    to_extract = ['weights.txt', 'centres.txt', 'tract_lengths.txt', 'average_orientations.txt', 'areas.txt',
                  'cortical.txt', 'hemisphere.txt']

    # get all files within root directory
    contents = os.listdir(parent)

    print(set(to_extract).difference(set(contents)))
    # if files are already extracted, exit the function
    if len(set(to_extract).difference(set(contents))) == 0:
        return []

    if len(set(to_extract).intersection(set(contents))) < len(to_extract):
        # open zip file
        archive = zipfile.ZipFile(path)

        # store all newly added files
        added = []

        # iterate over zip content and extract everything
        for ext in to_extract:
            if ext in archive.namelist():
                print(ext)
                # get filename
                new_filename = os.path.join(parent, ext)

                # extract file
                if not os.path.exists(new_filename):
                    archive.extract(ext, path=parent)
                    
                    # append newly added files
                    added.append(new_filename)
        return added

    return []

In [10]:
extract_zip("C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON04/ses-preop/SC.zip")

{'tract_lengths.txt', 'average_orientations.txt'}
weights.txt
centres.txt
tract_lengths.txt
average_orientations.txt
areas.txt
cortical.txt
hemisphere.txt


['C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON04/ses-preop/tract_lengths.txt',
 'C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON04/ses-preop/average_orientations.txt']

In [ ]:
'C:\Users\dinar\Desktop\gsoc_data\brain_tumor\sub-CON01\ses-preop\average_orientations.txt'.replace('')

In [14]:
import pandas as pd 
import csv 

def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.
    :param path:
    :return:
    """
    if path.split('.')[-1] not in ['txt', 'csv']:
        return

    try:
        file = pd.read_csv(path, index_col=None, header=None, sep=' ')
    except pd.errors.EmptyDataError:
        return 'remove'
    except pd.errors.ParserError:
        print(path)
    else:
        # if cortical.txt, hemisphere.txt, or areas.txt are present, return '\n' delimiter
        if path.endswith('hemisphere.txt') or path.endswith('cortical.txt') or path.endswith('areas.txt') \
                or path.endswith('times.txt'):
            file.to_csv(path, sep='\n', header=None, index=None)
            return '\n'

        # if file.shape[0] > 1 and file.shape[1] > 1:
        #     return '\s'
        
        # try with '\t' 
        trial =  pd.read_csv(path, header=None, sep='\t')
        if trial.shape[0] > 1 and trial.shape[1] > 1:
            return '\t'

        sniffer = csv.Sniffer()

        with open(path) as fp:
            try:
                delimiter = sniffer.sniff(fp.read(5000)).delimiter
            except Exception:
                delimiter = sniffer.sniff(fp.read(50)).delimiter

        delimiter = '\s' if delimiter == ' ' else delimiter
        return delimiter


In [15]:
find_separator('C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON01/ses-preop/centres.txt')

'\t'

In [13]:
trial =  pd.read_csv('C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON01/ses-preop/centres.txt', header=None, sep='\t')
trial.shape

(68, 4)

In [7]:
import sim2bids.utils
from sim2bids.generate import structure, subjects
from sim2bids.preprocess import preprocess as prep
from sim2bids.preprocess import prepare_inputs as prepare
from sim2bids.convert import convert, mat
from sim2bids.templates import templates as temp
from sim2bids.app import utils


convert.open_file('C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON01/ses-preop/centres.txt', 
                  subjects.find_separator('C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON01/ses-preop/centres.txt'))





,0,1,2,3
0,lh_bankssts,-55.206300,-42.3158,1.7060
1,lh_caudalanteriorcingulate,0.032388,4.3111,40.9250
2,lh_caudalmiddlefrontal,-33.510700,-3.3189,53.5018
3,lh_cuneus,-10.421100,-81.1302,3.9362
4,lh_entorhinal,-26.039400,5.5861,-20.3210
...,...,...,...,...
63,rh_supramarginal,52.684300,-45.7635,23.9020
64,rh_frontalpole,12.348800,61.2925,23.9876
65,rh_temporalpole,26.367000,22.6472,-10.1617
66,rh_transversetemporal,39.580800,-22.3416,12.5562


In [6]:
import pandas as pd

pd.read_csv('C:/Users/dinar/Desktop/gsoc_data/brain_tumor/sub-CON01/ses-preop/centres.txt', header=None, sep='\t')



,0,1,2,3
0,lh_bankssts,-55.206300,-42.3158,1.7060
1,lh_caudalanteriorcingulate,0.032388,4.3111,40.9250
2,lh_caudalmiddlefrontal,-33.510700,-3.3189,53.5018
3,lh_cuneus,-10.421100,-81.1302,3.9362
4,lh_entorhinal,-26.039400,5.5861,-20.3210
...,...,...,...,...
63,rh_supramarginal,52.684300,-45.7635,23.9020
64,rh_frontalpole,12.348800,61.2925,23.9876
65,rh_temporalpole,26.367000,22.6472,-10.1617
66,rh_transversetemporal,39.580800,-22.3416,12.5562


In [12]:
import pylems_py2xml
import os 
from sim2bids.app import app

pylems_py2xml.main.XML(inp=app.CODE, output_path=os.path.join(app.OUTPUT, 'param'), suffix=app.DESC, app=True)

In [10]:
import pylems_py2xml

In [5]:
import sim2bids.utils
from sim2bids.generate import structure, subjects
from sim2bids.preprocess import preprocess as prep
from sim2bids.preprocess import prepare_inputs as prepare
from sim2bids.convert import convert, mat
from sim2bids.templates import templates as temp
from sim2bids.app import utils


path = 'C:/Users/dinar/Desktop/50healthy_demo/centres.txt'

convert.open_file(path, subjects.find_separator(path))

,0,1,2,3
0,lh_bankssts,-50.1905,-38.6216,-10.6333
1,lh_caudalanteriorcingulate,-2.2099,16.7813,26.4645
2,lh_caudalmiddlefrontal,-33.7486,6.1286,39.3557
3,lh_cuneus,-10.3116,-81.3424,-7.9693
4,lh_entorhinal,-20.3763,11.2590,-38.4088
...,...,...,...,...
63,rh_supramarginal,50.3882,-33.9278,15.4574
64,rh_frontalpole,11.4958,72.6342,-1.0835
65,rh_temporalpole,26.8694,29.6034,-31.6738
66,rh_transversetemporal,40.0771,-15.6599,2.2133


In [5]:
from collections import OrderedDict

subs = OrderedDict([('sub-01', {'distances': {'name': 'distances', 'fname': 'AY_20111004_distances.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_distances.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'AY_20111004_weights.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_weights.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'emp_fc.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\emp_fc.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-bold_times_22min': {'name': 'alpha-speed100-G0.0325-bold_times_22min', 'fname': 'bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-bold_ts_22min': {'name': 'alpha-speed100-G0.0325-bold_ts_22min', 'fname': 'bold_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_ts_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-subs_times_22min': {'name': 'alpha-speed100-G0.0325-subs_times_22min', 'fname': 'subs_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\subs_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-subs_ts_22min': {'name': 'alpha-speed100-G0.0325-subs_ts_22min', 'fname': 'subs_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\subs_ts_22min.txt', 'ext': 'txt'}}), ('sub-02', {'distances': {'name': 'distances', 'fname': 'distances.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'emp_fc.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'weights.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_times': {'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'bold_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_ts': {'name': 'alpha-speed0.1-G10-bold_ts', 'fname': 'bold_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_times': {'name': 'alpha-speed0.1-G10-subs_times', 'fname': 'subs_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_ts': {'name': 'alpha-speed0.1-G10-subs_ts', 'fname': 'subs_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_times_22min': {'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'bold_times_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_ts_22min': {'name': 'alpha-speed100-G0.0398-bold_ts_22min', 'fname': 'bold_ts_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_times_22min': {'name': 'alpha-speed100-G0.0398-subs_times_22min', 'fname': 'subs_times_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_ts_22min': {'name': 'alpha-speed100-G0.0398-subs_ts_22min', 'fname': 'subs_ts_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-bold_times': {'name': 'delta-speed20-G0.1-bold_times', 'fname': 'bold_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_times': {'name': 'delta-speed20-G0.1-subs_times', 'fname': 'subs_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_ts': {'name': 'delta-speed20-G0.1-subs_ts', 'fname': 'subs_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_ts.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-bold_times': {'name': 'delta-speed20-G0.2-bold_times', 'fname': 'bold_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_times': {'name': 'delta-speed20-G0.2-subs_times', 'fname': 'subs_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_ts': {'name': 'delta-speed20-G0.2-subs_ts', 'fname': 'subs_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_ts.txt', 'ext': 'txt'}})])


In [7]:
from sim2bids.app import app
from sim2bids.app import utils

app.ALL_FILES

In [8]:
files = utils.get_content('C:/Users/dinar/Desktop/50healthy_demo/', '')
files

['C:/Users/dinar/Desktop/50healthy_demo/50healthy_code.py',
 'C:/Users/dinar/Desktop/50healthy_demo/centres.txt',
 'C:/Users/dinar/Desktop/50healthy_demo/subsample_times.tsv',
 'C:/Users/dinar/Desktop/50healthy_demo/times.tsv',
 'C:/Users/dinar/Desktop/50healthy_demo/.idea\\.gitignore',
 'C:/Users/dinar/Desktop/50healthy_demo/.idea\\50healthy_demo.iml',
 'C:/Users/dinar/Desktop/50healthy_demo/.idea\\misc.xml',
 'C:/Users/dinar/Desktop/50healthy_demo/.idea\\modules.xml',
 'C:/Users/dinar/Desktop/50healthy_demo/.idea\\workspace.xml',
 'C:/Users/dinar/Desktop/50healthy_demo/.idea\\inspectionProfiles\\profiles_settings.xml',
 'C:/Users/dinar/Desktop/50healthy_demo/.idea\\inspectionProfiles\\Project_Default.xml',
 'C:/Users/dinar/Desktop/50healthy_demo/AA_20120815\\distances.txt',
 'C:/Users/dinar/Desktop/50healthy_demo/AA_20120815\\emp_fc.txt',
 'C:/Users/dinar/Desktop/50healthy_demo/AA_20120815\\weights.txt',
 'C:/Users/dinar/Desktop/50healthy_demo/AA_20120815\\series_alpha\\cspeed0.1\\cs

In [13]:
from pathlib import Path

files = [Path(f) for f in files if '/.' not in f]
files

[WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/50healthy_code.py'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/centres.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/subsample_times.tsv'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/times.tsv'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/distances.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/emp_fc.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/weights.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/series_alpha/cspeed0.1/csf 10/bold_times.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/series_alpha/cspeed0.1/csf 10/bold_ts.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/series_alpha/cspeed0.1/csf 10/subs_times.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/series_alpha/cspeed0.1/csf 10/subs_ts.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50h

In [17]:


sub_paths = []

for k, v in subs.items():
    for k2, v2 in v.items():
        if Path(v2['path']) in files:
            sub_paths.append(Path(v2['path']))

In [18]:
sub_paths

[WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/AY_20111004_distances.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/AY_20111004_weights.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/emp_fc.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/series_alpha/cspeed100/csf 0.0325/bold_times_22min.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/series_alpha/cspeed100/csf 0.0325/bold_ts_22min.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/series_alpha/cspeed100/csf 0.0325/subs_times_22min.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/series_alpha/cspeed100/csf 0.0325/subs_ts_22min.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/distances.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/emp_fc.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/weights.txt'),
 WindowsPath('C:/U

In [19]:
diff = set(files).difference(set(sub_paths))
diff

{WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/50healthy_code.py'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/series_delta/cspeed20/csf 0.1/arr.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/AA_20120815/series_delta/cspeed20/csf 0.2/arr.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/centres.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/subsample_times.tsv'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/times.tsv')}

In [26]:
import os

diff = os.listdir(Path('C:/Users/dinar/Desktop/50healthy_demo'))
diff = [Path(f'C:/Users/dinar/Desktop/50healthy_demo/{p}') for p in diff if not os.path.isdir(Path(f'C:/Users/dinar/Desktop/50healthy_demo/{p}'))]

In [27]:
diff

[WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/50healthy_code.py'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/centres.txt'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/subsample_times.tsv'),
 WindowsPath('C:/Users/dinar/Desktop/50healthy_demo/times.tsv')]

OrderedDict([('sub-01', {'distances': {'name': 'distances', 'fname': 'AY_20111004_distances.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_distances.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'AY_20111004_weights.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_weights.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'emp_fc.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\emp_fc.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-bold_times_22min': {'name': 'alpha-speed100-G0.0325-bold_times_22min', 'fname': 'bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_times_22min.txt', 'ext': 'txt'}, 'alpha-speed1

results > 1
result[0]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed0.1\csf 10\bold_times.txt IS NOT SIMILAR to result[idx]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed0.1\csf 10\subs_times.txt

result[0]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed0.1\csf 10\bold_times.txt IS NOT SIMILAR to result[idx]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed100\csf 0.0398\bold_times_22min.txt

result[0]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed0.1\csf 10\bold_times.txt IS NOT SIMILAR to result[idx]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed100\csf 0.0398\subs_times_22min.txt

result[0]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed0.1\csf 10\bold_times.txt IS SIMILAR to result[idx]=C:\Users\dinar\Desktop\50healthy_demo\AY_20111004\series_alpha\cspeed100\csf 0.0325\bold_times_22min.txt

result[0]=C:\Users\dinar\Desktop\50hea

result[0]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed0.1\csf 10\bold_times.txt IS NOT SIMILAR to result[idx]=C:\Users\dinar\Desktop\50healthy_demo\AY_20111004\series_alpha\cspeed100\csf 0.0325\subs_times_22min.txt

result[0]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed0.1\csf 10\bold_times.txt IS NOT SIMILAR to result[idx]=C:\Users\dinar\Desktop\50healthy_demo\AY_20111004\series_alpha\cspeed100\csf 0.0325\subs_ts_22min.txt

TIMES: speed
RHYTHM: delta
RESULTS @435: ['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\arr.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_ts.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\seri

ALL TIMES: ['bold_times_22min.txt', 'desc-default_delta-bold_times.txt.tsv', 'desc-default_delta-subs_times.txt.tsv', 'subsample_times.tsv', 'times', 'desc-default_delta-subs_times.txt.json', 'desc-default_delta-bold_times.txt.json', 'subs_times_22min.txt', 'desc-default_alpha-subs_times.json', 'desc-default_alpha-bold_times.json', 'speed', 'subs_times.txt', 'desc-default_alpha-subs_times.tsv', 'desc-default_alpha-bold_times.tsv', 'bold_times.txt']
TIMES: bold_times_22min.txt
RHYTHM: alpha
RESULTS @435: ['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_times_22min.txt']
results > 1
result[0]=C:\Users\dinar\Desktop\50healthy_demo\AA_20120815\series_alpha\cspeed100\csf 0.0398\bold_times_22min.txt IS NOT SIMILAR to result[idx]=C:\Users\dinar\Desktop\50healthy_demo\AY_20111004\series_alpha\cspeed100\csf 0.0325\bold_times_22mi

In [ ]:
for file in diff:
    if file.endswith('.py'):
        app.CODE = file
    else:
        folders = 

In [5]:
from scipy.io import loadmat

In [6]:
loadmat("C:/Users/dinar/Downloads/AA_20120815_SC.mat_SC_log_SC_22min_csf_0.0328.mat")

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Feb 14 04:52:19 2017',
 '__version__': '1.0',
 '__globals__': [],
 'Bold_time': array([[5.0000e+02, 1.0000e+03, 1.5000e+03, ..., 1.3190e+06, 1.3195e+06,
         1.3200e+06]]),
 'Bold_data': array([[[[ -141.48949614],
          [  -62.71886885],
          [  -82.33999015],
          ...,
          [  167.00314701],
          [   19.36960773],
          [   -9.46151917]]],
 
 
        [[[ -143.08279113],
          [  -64.29409459],
          [  -83.94336549],
          ...,
          [  165.27042134],
          [   17.67803488],
          [  -11.21377617]]],
 
 
        [[[ -153.07308041],
          [  -74.0562326 ],
          [  -94.2439947 ],
          ...,
          [  154.36469672],
          [    7.11115482],
          [  -22.63011437]]],
 
 
        ...,
 
 
        [[[-1464.41279874],
          [-1354.23103094],
          [-1532.3469634 ],
          ...,
          [-1294.22347323],
          [-1373.99152578],
 

In [5]:
from collections import OrderedDict

subs = OrderedDict([('sub-01', {'distances': {'name': 'distances', 'fname': 'AY_20111004_distances.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_distances.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'AY_20111004_weights.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_weights.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'emp_fc.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\emp_fc.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-bold_times_22min': {'name': 'alpha-speed100-G0.0325-bold_times_22min', 'fname': 'bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-bold_ts_22min': {'name': 'alpha-speed100-G0.0325-bold_ts_22min', 'fname': 'bold_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_ts_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-subs_times_22min': {'name': 'alpha-speed100-G0.0325-subs_times_22min', 'fname': 'subs_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\subs_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0325-subs_ts_22min': {'name': 'alpha-speed100-G0.0325-subs_ts_22min', 'fname': 'subs_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\subs_ts_22min.txt', 'ext': 'txt'}}), ('sub-02', {'distances': {'name': 'distances', 'fname': 'distances.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'emp_fc.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'weights.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_times': {'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'bold_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_ts': {'name': 'alpha-speed0.1-G10-bold_ts', 'fname': 'bold_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_times': {'name': 'alpha-speed0.1-G10-subs_times', 'fname': 'subs_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_ts': {'name': 'alpha-speed0.1-G10-subs_ts', 'fname': 'subs_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_times_22min': {'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'bold_times_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_ts_22min': {'name': 'alpha-speed100-G0.0398-bold_ts_22min', 'fname': 'bold_ts_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_times_22min': {'name': 'alpha-speed100-G0.0398-subs_times_22min', 'fname': 'subs_times_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_ts_22min': {'name': 'alpha-speed100-G0.0398-subs_ts_22min', 'fname': 'subs_ts_22min.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-bold_times': {'name': 'delta-speed20-G0.1-bold_times', 'fname': 'bold_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_times': {'name': 'delta-speed20-G0.1-subs_times', 'fname': 'subs_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_ts': {'name': 'delta-speed20-G0.1-subs_ts', 'fname': 'subs_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_ts.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-bold_times': {'name': 'delta-speed20-G0.2-bold_times', 'fname': 'bold_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_times': {'name': 'delta-speed20-G0.2-subs_times', 'fname': 'subs_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_ts': {'name': 'delta-speed20-G0.2-subs_ts', 'fname': 'subs_ts.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_ts.txt', 'ext': 'txt'}})])

In [8]:
import sim2bids.templates.templates as temp
import sim2bids.app.app as app
from sim2bids.convert import convert
import pandas as pd

TIMES_TO_SKIP = []


def traverse_times(sub, folders, ses):
    global TIMES_TO_SKIP
    
    # get description
    desc = temp.file_desc['times'] if 'bold' not in sub['name'] else temp.file_desc['bold_times']
    similar = False

    # check if times are similar
    for times in app.TIMES:
        for rhythm in ['alpha', 'delta', 'beta', 'gamma', 'theta']:
            # check if the file needs to be skipped
            if f'{rhythm}_{times}' in TIMES_TO_SKIP:
                return
            
            
            results = convert.get_specific(times + '.', rhythm)
            
            if len(results) > 0:
                open_df = lambda x: pd.read_csv(x, header=None, sep='\t')
                first = open_df(results[0])
                
                if len(results) > 1:
                    for result in results[1:]:
                        if first.equals(open_df(result)):
                            similar = True
                            TIMES_TO_SKIP.append(f'{rhythm}_{times}')
                            
                            # save in global folder
                            save_files(sub, os.path.join(app.OUTPUT, 'coord'), first, 'default', desc=desc)
                            
                else:
                    similar = True
                    TIMES_TO_SKIP.append(f'{rhythm}_{times}')
                    # save in global folder
                    save_files(sub, os.path.join(app.OUTPUT, 'coord'), first, 'default', desc=desc)
   


    
    
#     ftype = sub['name'].split('-')[-1]
    
#     results = convert.get_specific(ftype + '.', 'alpha')
#     print(results)
    
#     if len(results) == 0:
#         return
#     else:
#         open_df = lambda x: pd.read_csv(x, header=None, sep='\t')
#         first = open_df(results[0])
        
#         if len(results) > 1:
#             for result in results[1:]:
#                 if first.equals(open_df(result)):
#                     similar = True
#         else:
#             similar = True
        
    
    

In [9]:
traverse_times({'name': 'delta-speed20-G0.2-bold_times', 'fname': 'bold_times.txt', 'sid': 'sub-02', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt', 'ext': 'txt'}, None, None)

TypeError: 'NoneType' object is not iterable

In [10]:
app.ALL_FILES

In [11]:
import sim2bids.app.app as app

app.TIMES

['subs_times_22min.txt',
 'bold_times.txt',
 'subs_times.txt',
 'bold_times_22min.txt']

In [12]:
app.ALL_FILES

In [9]:
import sim2bids.app.app as app

def get_specific(filetype: str, constraint: str = None) -> list:
    """
    Get all files that correspond to the filetype. For example,
    if filetype is equal to "areas", this function will return
    all files containing that keyword.

    Parameters
    ----------
    filetype: str
        Type of the file to search for. Accepted types:
            'weight', 'distance', 'tract_length', 'delay', 'speed',                 # Network (net)
            'nodes', 'labels', 'centres', 'area', 'hemisphere', 'cortical',         # Coordinates (coord)
            'orientation', 'average_orientation', 'normal', 'times', 'vertices',    # Coordinates (coord)
            'faces', 'vnormal', 'fnormal', 'sensor', 'map', 'volume',               # Coordinates (coord)
            'cartesian2d', 'cartesian3d', 'polar2d', 'polar3d',                     # Coordinates (coord)
            'vars', 'stimuli', 'noise', 'spike', 'raster', 'ts', 'event', 'emp'     # Timeseries (ts)
            'fc'                                                                    # Spatial (spatial)

    Returns
    -------
        Returns a list of all files that end with the specified filetype.
        :param constraint:
    """

    # create emtpy list to store appropriate files
    content = []

    # iterate over all files found by the app previously
    for file in app.ALL_FILES:
        # check if the keyword is present
        if filetype in file:
            print(file)
            if constraint is not None and constraint in file:
                content.append(file)
            if constraint is None:
                # if yes, append the value
                content.append(file)

    # return the list of newly-found files
    return content

In [10]:
get_specific('bold_times', 'alpha')

TypeError: 'NoneType' object is not iterable

In [6]:
from sim2bids.generate import subjects
import os

subjects.find_matches(os.listdir('C:/Users/dinar/Desktop/50healthy_demo'))

['AA_20120815', 'AY_20111004']

multi-subject with matches
AY_20111004 ['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_distances.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\AY_20111004_weights.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_times_22min.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\bold_ts_22min.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\subs_times_22min.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\subs_ts_22min.txt']


In [6]:
subs = {'sub-01': {'path': 'path'}, 'sub-02': {}, 'sub-03': {}, 'sub-04': {'path': 'path'}}

In [41]:
from collections import OrderedDict

subs = {'sub-01': {}, 'sub-02': {}, 'sub-03': {}, 'sub-04': {'path': 'path'}}


def create_sid_sub(sid=None, subs=None):
    check_empty(subs)
    return prep.
    

def check_empty(subs):
    temp, to_delete = OrderedDict(), []
    
    for k, v in subs.items():
        if len(v) > 0:
            temp[k] = v
    
    # iterate over existing ids and correct indexing if necessary
    for i, (k, v) in enumerate(temp.items(), start=1):
        if not k.endswith(str(i)):
            idx = f'sub-0{i}' if i < 10 else f'sub-{i}'
            to_delete.append(k)
            temp[idx] = v
    
    
    for idx in to_delete:
        del temp[idx]
    
    subs = temp
                

In [42]:
create_sid_sub(subs=subs)

OrderedDict([('sub-01', {'path': 'path'})])


In [5]:
import sim2bids.generate.subjects as subs

subs.get_name('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/series_alpha/cspeed100/csf 0.0325/AY_20111004_SC_new.mat_SC_log_SC_22min_csf_0.0325.mat')




'C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/series_alpha/cspeed100/csf 0.0325/AY_20111004_SC_new.mat_SC_log_SC_22min_csf_0.0325.mat'

In [14]:
import os
import re
import sim2bids.app.app as app


def get_name(path):
    # get file's basename (=without root directory)
    base = os.path.basename(path)
    print(base)

    # instantiate speed, global coupling, and rhythms
    speed, g, series = None, None, None

    # iterate over rhythms if exist to get naming for the file
    for s in ['alpha', 'delta', 'gamma', 'theta', 'beta']:
        if s in path.lower():
            speed_temp = re.findall(r'speed[0-9\.]+', path)
            series = s

            if len(speed_temp) > 0:
                speed = speed_temp[0]

            g_temp = re.findall(r'csf\s[0-9\.]+', path)

            if len(g_temp) > 0:
                g = g_temp[0].replace('csf', '').strip()

    print(base.split('.')[0])            
    name = accepted(base.split('.')[0])
    print(name)

    if name is False:
        return base

    if series is not None and speed is not None and g is not None:
        return f'{series}-{speed}-G{g}-{name[-1]}'
    elif series is not None and speed is None and g is None:
        return f'{series}'

    if name[-1] in [*app.ACCEPTED[:4], *app.ACCEPTED[7:9], *app.ACCEPTED[11:13], *app.ACCEPTED[17:20],
                    'spike', 'event']:
        return name[-1] + 's'

    return name[-1]


def accepted(name, return_accepted=False):
    for accept in app.ACCEPTED:
        if accept in name:
            if accept == 'ts' or accept == 'times':
                split = name.split('_')
                if len(split) == 2:
                    if return_accepted:
                        return accept
                    return True, split[-2] + '_' + split[-1]
                if len(split) >= 3:
                    if return_accepted:
                        return accept
                    return True, split[-3] + '_' + split[-2] + '_' + split[-1]

                if return_accepted:
                    return accept

                return True, accept

            if accept == 'emp' and 'emp_fc' in name:
                if return_accepted:
                    return accept
                return True, 'emp_fc'
            if accept != 'ts':
                if return_accepted:
                    return accept
                return True, accept
    return False

In [15]:
get_name('C:/Users/dinar/Desktop/50healthy_demo/AY_20111004/series_alpha/cspeed100/csf 0.0325/bold_times_22min')



bold_times_22min
bold_times_22min
(True, 'bold_times_22min')


'alpha-speed100-G0.0325-bold_times_22min'

In [5]:
import os
import re
import shutil

import numpy as np
import pandas as pd

import panel as pn
import scipy.io as sio
import mat73
from sim2bids.app import app
from sim2bids.generate import subjects


def save_mat(sub, og_path, extract=False):
    file = traverse_file(sub['path'])
    root = os.path.dirname(sub['path'])
    multi_folder, extracted = og_path == root, False

    if extract:
        new_files = []

        # extract files
        if file is not None:
            # first, check if there are matches according to unique ID
            # find matches for unique patients and transfer files there,
            # e.g., weights and distances
            # transfer_files(sub, og_path)

            # iterate over matlab files and extract files
            for k in file.keys():
                if not k.startswith('__') and type(file[k]) in [np.ndarray, list]:
                    f = file[k]

                    if len(file[k].shape) == 4:
                        f = file[k][:, 0, :, 0]

                    if len(file[k].shape) == 3:
                        f = file[k][:, 0, :]

                    name = check_name(k)
                    minutes = re.findall(r'[0-9]+min', sub['path'])
                    name = f'{name}_{minutes[0]}' if len(minutes) > 0 else name

                    if multi_folder:
                        path = os.path.join(os.path.join(root, sub['sid']), name + '.txt')
                    else:
                        path = os.path.join(root, name + '.txt')

                    if not os.path.exists(path):
                        extracted = True
                        new_files.append(name + '.txt')
                        pd.DataFrame(f).to_csv(path, index=None, header=None, sep='\t')

            if extracted:
                # delete mat file
                os.remove(sub['path'])

        return new_files


def transfer_files(sub, og_path):
    """
    Iterate over folder and find unique ID's corresponding files
    """
    root = os.path.dirname(sub['path'])

    match = subjects.find_matches([sub['path']])
    sid, folder = sub['sid'], os.path.join(root, sub['sid'])

    if len(match) > 0:
        if not os.path.exists(folder):
            os.mkdir(folder)

        for f in os.listdir(og_path):
            if os.path.isdir(os.path.join(og_path, f)) or f.endswith('mat'):
                continue
            if match[0] in f:
                basename = f.replace(match[0], '').strip('_')
                shutil.move(os.path.join(og_path, f), os.path.join(og_path, sub['sid'], basename))


def check_name(name):
    name = subjects.get_name(name.lower())

    if 'time' in name:
        if 'times' in name:
            return name
        return name.replace('time', 'times')
    if 'data' in name:
        return name.replace('data', 'ts')
    return name


def save_mat73(subs, folders, ses):
    return mat73.loadmat(subs['path'])


def traverse_file(path):
    name = os.path.basename(path)

    try:
        mat = sio.loadmat(path, squeeze_me=True)
    except NotImplementedError:
        return mat73.loadmat(path)
    except sio.matlab._miobase.MatReadError:
        pn.state.notifications.error(f'File `{name}` is empty! Aborting...')
        return None
    else:
        return mat


In [ ]:
save_mat({'path': })

In [8]:
def get_name(path):
    # get file's basename (=without root directory)
    base = os.path.basename(path)

    # instantiate speed, global coupling, and rhythms
    speed, g, series = None, None, None

    # iterate over rhythms if exist to get naming for the file
    for s in ['alpha', 'delta', 'gamma', 'theta', 'beta']:
        if s in path.lower():
            speed_temp = re.findall(r'speed[0-9\.]+', path)
            series = s

            if len(speed_temp) > 0:
                speed = speed_temp[0]

            g_temp = re.findall(r'csf\s[0-9\.]+', path)

            if len(g_temp) > 0:
                g = g_temp[0].replace('csf', '').strip()

    name = accepted(base.split('.')[0])

    if name is False:
        return name

    if series is not None and speed is not None and g is not None:
        return f'{series}-{speed}-G{g}-{name[-1]}'
    elif series is not None and speed is None and g is None:
        return f'{series}'

    if name[-1] in [*app.ACCEPTED[:4], *app.ACCEPTED[7:9], *app.ACCEPTED[11:13], *app.ACCEPTED[17:20],
                    'spike', 'event']:
        return name[-1] + 's'

    return name[-1]


def accepted(name, return_accepted=False):
    for accept in app.ACCEPTED:
        if accept in name:
            if accept == 'ts' or accept == 'times':
                split = name.split('_')
                if len(split) >= 2:
                    if return_accepted:
                        return accept
                    if 'min' in name:
                        return True, split[-3] + '_' + split[-2] + '_' + split[-1]
                    return True, split[-2] + '_' + split[-1]

                if return_accepted:
                    return accept

                return True, accept

            if accept == 'emp' and 'emp_fc' in name:
                if return_accepted:
                    return accept
                return True, 'emp_fc'
            if accept != 'ts':
                if return_accepted:
                    return accept
                return True, accept
    return False

In [11]:
get_name('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt')

'delta-speed20-G0.1-subs_times'

In [15]:
import os
import csv
import re
from pathlib import Path

import numpy as np
import pandas as pd

from sim2bids.app import app
from collections import OrderedDict
import sim2bids.preprocess.preprocess as prep
from sim2bids.convert import convert
from sim2bids.app import utils


class Files:
    def __init__(self, path, files):
        self.path = path
        self.files = files
        self.subs = OrderedDict()

        # get all files' absolute paths
        self.content = utils.get_content(path, files)

        # get all files
        app.ALL_FILES = self.content

        # get all files' unique names
        self.basename = set(utils.get_content(path, files, basename=True))

        # check for multi-subject in one folder
        self.match = find_matches(self.basename)

        # check if the input is for single-subject or multi-subject
        app.MULTI_INPUT = self.check_input()

        # define a variable that is going to check whether input
        # contains sessions-based subject. Sessions include `ses-preop` and 'ses-postop'
        self.ses_found = False

        # traverse folders
        self.traverse_files()

    def check_input(self):
        # weights are the essential files for each subject,
        # therefore, finding the number of weights will reveal
        # whether the file folder is multi-folder or not
        basename = [os.path.basename(x) for x in self.content]
        weights = 0

        for name in basename:
            if 'weight' in name:
                weights += 1
        return False if weights == 1 else True

    def traverse_files(self):
        # if the whole folder is passed, open that folder
        path, files, changed = self.path, self.files, False
        
        if len(files) == 1 and os.path.isdir(os.path.join(path, files[0])) and files[0] not in ['ses-preop',
                                                                                                'ses-postop']:
            path = os.path.join(path, files[0])
            files = os.listdir(path)

        # traverse multi-subject inputs
        if app.MULTI_INPUT:
            # traverse multi-subject in one folder structure
            if len(self.match) > 0:
                for k, v in get_unique_subs(self.match, self.content).items():
                    # create a new ID
                    sid = self.create_sid_sub()
                    self.subs[sid].update(prepare_subs([os.path.join(path, x) for x in v], sid))
            else:

                for file in files:
                    if not os.path.isdir(os.path.join(path, file)):
                        continue
                    
                    if file.endswith('.py'):
                        app.CODE = file
                    
                    sid = self.create_sid_sub()

                    # Step 5: get all content
                    all_files = os.listdir(path)

                    # Step 6: traverse ses-preop if present
                    if 'ses-preop' in all_files:
                        self.save_sessions('ses-preop', all_files, sid, path)

                    # Step 7: traverse ses-postop if present
                    if 'ses-postop' in all_files:
                        self.save_sessions('ses-postop', all_files, sid, path)

                    if 'ses-preop' not in all_files and 'ses-postop' not in all_files:
                        if os.path.basename(path) == file:
                            self.subs[sid] = prepare_subs([*utils.get_content(path.replace(file, ''), file), 
                                                           *self.get_extra_files(all_files, path)], sid)
                        else:
                            self.subs[sid] = prepare_subs([*utils.get_content(path, file), 
                                                           *self.get_extra_files(all_files, path)], sid)

        else:
            # check if there are no folders inside
            sid = self.create_sid_sub()
            self.save_sessions('ses-preop', files, sid, os.path.join(path, 'ses-preop'))
            self.save_sessions('ses-postop', files, sid, os.path.join(path, 'ses-postop'))

            if not self.ses_found:
                self.create_sid_sub(sid)
                self.subs[sid] = prepare_subs(utils.get_content(path, files), sid)
        
        prep.reset_index()

    def get_extra_files(self, files, path):
        return [os.path.join(path, file) for file in files if not os.path.isdir(os.path.join(path, file))]
        
    
    def save_sessions(self, ses, files, sid, path):
        if ses in files:
            self.ses_found = True
            if sid not in self.subs.keys():
                self.subs[sid] = OrderedDict()
            if ses not in self.subs[sid].keys():
                self.subs[sid][ses] = OrderedDict()

            self.subs[sid][ses].update(prepare_subs(utils.get_content(path, ses), sid))

    def create_sid_sub(self, sid=None):
        if sid is None:
            if len(self.subs) > 0:
                for k, v in self.subs.items():
                    if len(v) == 0:
                        return k
                    return prep.create_uuid()
            else:
                sid = prep.create_uuid()
                return sid

        # create a dictionary to store values
        if sid not in self.subs.keys():
            self.subs[sid] = {}

        return sid


def traverse_single(path, selected, sid, ses=None):
    if ses is not None:
        return prepare_subs(utils.get_content(path, ses), sid)
    else:
        return prepare_subs(utils.get_content(path, selected), sid)


def find_matches(paths):
    unique_ids = []

    for path in paths:
        match = re.findall('[A-Z]{2,3}_[0-9]{2,}', path)
        if len(match) > 0 and not path.endswith('.h5'):
            unique_ids.append(match[0])

    return list(set(unique_ids))


def get_unique_subs(match, contents):
    subs = OrderedDict()

    for idx in range(len(match)):
        subs[match[idx]] = [x for x in contents if match[idx] in x]

    return subs


def prepare_subs(file_paths, sid):
    subs = {}

    for file_path in file_paths:
        name = get_filename(file_path)

        if file_path.endswith('.h5'):
            name = name.split('_')[0] + '.h5'
        else:
            if 'bold' not in name or 'emp' not in name or 'times' not in name:
                name = name.split('_')[-1] if '_' in name else name
        desc = app.DESC

        # get extensions
        ext = os.path.basename(file_path).split('.')[-1]

        # check if file ends with CSV or dat, if true, change file
        # extension to plain TXT. It's necessary so that there's minimal
        # number of "if" statements in the future traversals
        if ext.endswith('csv') or ext.endswith('dat'):
            # instantiate a new path
            new_path = file_path.replace(ext, 'txt')

            if os.path.exists(file_path):
                p = Path(file_path)
                p.rename(p.with_suffix('.txt'))

                # set the new path
                file_path = new_path

        # rename tract_lengths to distances
        if name == 'tract_lengths.txt':
            name = 'distances.txt'

        if not os.path.exists(file_path) and 'distances' in file_path:
            if os.path.exists(file_path.replace('distances', 'tract_lengths')):
                os.replace(file_path.replace('distances', 'tract_lengths'), file_path)
            else:
                continue

        # rename tract_lengths to distances in the physical folder location
        if 'tract_lengths' in file_path:
            new_path = file_path.replace('tract_lengths', 'distances')
            os.replace(file_path, new_path)
            file_path = new_path

        # rename average_orientations to normals in both subject- and physical levels
        if name in ['average_orientations.txt', 'orientations.txt']:
            name = 'normals.txt'
            new_path = file_path.replace(name.replace('.txt', ''), 'normals')
            os.replace(file_path, new_path)
            file_path = new_path

        # check if separator is missing, if so remove the file entirely
        name = get_name(file_path)

        if name is False:
            continue
        else:
            sep = find_separator(file_path)

            if sep == 'remove':
                os.remove(file_path)
                continue

            subs[name] = {
                'name': name,
                'fname': file_path.split('/')[-1],
                'sid': sid,
                'desc': desc,
                'sep': sep,
                'path': file_path,
                'ext': get_file_ext(file_path),
            }

            if subs[name]['name'] in ['centres', 'centre']:
                app.CENTERS = True

            # save network path
            if len(convert.NETWORK) < 2:
                desc, fname = app.DESC, name.split('.')[0]

                if fname in ['weights', 'distances']:
                    if 'ses-preop' in subs[name]['path'] in subs[name]['path']:
                        convert.NETWORK.append(f'../{sid}/ses-preop/net/{sid}_desc-{app.DESC}_{fname}.json')
                    elif 'ses-postop' in subs[name]['path'] in subs[name]['path']:
                        convert.NETWORK.append(f'../{sid}/ses-postop/net/{sid}_desc-{app.DESC}_{fname}.json')
                    else:
                        convert.NETWORK.append(f'../{sid}/net/{sid}_desc-{app.DESC}_{fname}.json')

                    convert.NETWORK = list(set(convert.NETWORK))

    return subs


def get_name(path):
    # get file's basename (=without root directory)
    base = os.path.basename(path)

    # instantiate speed, global coupling, and rhythms
    speed, g, series = None, None, None

    # iterate over rhythms if exist to get naming for the file
    for s in ['alpha', 'delta', 'gamma', 'theta', 'beta']:
        if s in path.lower():
            speed_temp = re.findall(r'speed[0-9\.]+', path)
            series = s

            if len(speed_temp) > 0:
                speed = speed_temp[0]

            g_temp = re.findall(r'csf\s[0-9\.]+', path)

            if len(g_temp) > 0:
                g = g_temp[0].replace('csf', '').strip()

    name = accepted(base.split('.')[0])

    if name is False:
        return name

    if series is not None and speed is not None and g is not None:
        return f'{series}-{speed}-G{g}-{name[-1]}'
    elif series is not None and speed is None and g is None:
        return f'{series}'

    if name[-1] in [*app.ACCEPTED[:4], *app.ACCEPTED[7:9], *app.ACCEPTED[11:13], *app.ACCEPTED[17:20],
                    'spike', 'event']:
        return name[-1] + 's'

    return name[-1]


def accepted(name, return_accepted=False):
    for accept in app.ACCEPTED:
        if accept in name:
            if accept == 'ts' or accept == 'times':
                split = name.split('_')
                if len(split) >= 2:
                    if return_accepted:
                        return accept
                    if 'min' in name:
                        return True, split[-3] + '_' + split[-2] + '_' + split[-1]
                    return True, split[-2] + '_' + split[-1]

                if return_accepted:
                    return accept

                return True, accept

            if accept == 'emp' and 'emp_fc' in name:
                if return_accepted:
                    return accept
                return True, 'emp_fc'
            if accept != 'ts':
                if return_accepted:
                    return accept
                return True, accept
    return False


def get_filename(path):
    return os.path.basename(path)


def get_file_ext(path):
    return path.split('.')[-1]


def find_separator(path):
    """
    Find the separator/delimiter used in the file to ensure no exception
    is raised while reading files.
    :param path:
    :return:
    """
    if path.split('.')[-1] not in ['txt', 'csv']:
        return

    try:
        file = pd.read_csv(path, index_col=None, header=None, sep=' ')
    except pd.errors.EmptyDataError:
        return 'remove'
    except pd.errors.ParserError:
        print(path)

    # if cortical.txt, hemisphere.txt, or areas.txt are present, return '\n' delimiter
    if path.endswith('hemisphere.txt') or path.endswith('cortical.txt') or path.endswith('areas.txt') \
            or path.endswith('times.txt'):
        file.to_csv(path, sep='\n', header=None, index=None)
        return '\n'

    if file.shape[0] > 1 and file.shape[1] > 1:
        return '\s'

    sniffer = csv.Sniffer()

    with open(path) as fp:
        try:
            delimiter = sniffer.sniff(fp.read(5000)).delimiter
        except Exception:
            delimiter = sniffer.sniff(fp.read(50)).delimiter

    delimiter = '\s' if delimiter == ' ' else delimiter
    return delimiter


In [16]:
res = Files(app.INPUT, os.listdir(app.INPUT))

In [17]:
res.subs

OrderedDict([('sub-01',
              {'bold_times_22min': {'name': 'bold_times_22min',
                'fname': 'inputs\\1\\bold_times_22min.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '.',
                'path': 'inputs\\1\\bold_times_22min.txt',
                'ext': 'txt'},
               'bold_ts_22min': {'name': 'bold_ts_22min',
                'fname': 'inputs\\1\\bold_ts_22min.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
                'path': 'inputs\\1\\bold_ts_22min.txt',
                'ext': 'txt'},
               'distances': {'name': 'distances',
                'fname': 'inputs\\1\\distances.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
                'path': 'inputs\\1\\distances.txt',
                'ext': 'txt'},
               'subs_times_22min': {'name': 'subs_times_22min',
                'fna

In [10]:
import os
import shutil
from sim2bids.generate.subjects import find_matches
from sim2bids.app.utils import get_content
from sim2bids.convert import mat 


def preprocess(path, files, input_path):
    # set to true if there were changes made
    changed = False 
    
    # create directory if doesn't exist
    if not os.path.exists(input_path):
        os.mkdir(input_path)
    
    # check if the "files" is a single directory, if so
    # change the path to that folder storing all files 
    if len(files) == 1 and os.path.isdir(path, files[0]):
        path = os.path.join(path, files[0])
    
    # iterate over contents and see which input type is passed
    # 1. get all files 
    content = get_content(path, files) if len(files) > 1 else get_content(path, '')
    
    # 2. check for unique IDs 
    matches = find_matches(content)
    
    if len(matches) > 0:
        changed = True
        transfer_files(content, matches, path, input_path)

        
def transfer_files(content, matches, path, input_path):
    transferred, mat_files = [], {}
    sid = 1
    
    def get_files(con, match):
        return [c for c in con if match in c]
    
    for match in matches:
        files = get_files(content, match)
        
        # create directory if doesn't exist
        p = os.path.join(input_path, str(sid))
        
        if not os.path.exists(p):
            os.mkdir(p)
        
        for file in files:
            if file.endswith('mat'):
                mat_files[str(sid)] = {'path': os.path.join(input_path, str(sid), os.path.basename(file).replace(match, '').strip('_,.\s')),
                                  'sid': str(sid)}
            
            transferred.append(file)
            shutil.copy(file, os.path.join(input_path, str(sid), os.path.basename(file).replace(match, '').strip('_,.\s')))
        
        sid += 1
    
    # transfer global files
    for file in list(set(content).difference(transferred)):
        shutil.copy(file, input_path)
    
    # extract content from matlab files
    for k, v in mat_files.items():
        mat.save_mat(mat_files[k], os.path.dirname(file), extract=True)



In [11]:
preprocess('C:/Users/dinar/Desktop/50healthy_test', '', 'inputs')

{'path': 'inputs\\1\\SC.mat_SC_log_SC_22min_csf_0.0328.mat', 'sid': '1'}
{'path': 'inputs\\2\\SC_new.mat_SC_log_SC_22min_csf_0.0325.mat', 'sid': '2'}


In [20]:
def transfer_files(sub, og_path):
    """
    Iterate over folder and find unique ID's corresponding files
    """
    root = os.path.dirname(sub['path'])
    
    match = subjects.find_matches([sub['path']])
    sid, folder = sub['sid'], os.path.join(root, sub['sid'])

    if len(match) > 0:
        if not os.path.exists(folder):
            os.mkdir(folder)

        for f in os.listdir(og_path):
            if os.path.isdir(os.path.join(og_path, f)) or f.endswith('mat'):
                continue
            if match[0] in f:
                shutil.move(os.path.join(og_path, f), os.path.join(og_path, sub['sid']))

In [21]:
from sim2bids.generate import subjects
from sim2bids.convert import mat
import os
import shutil

transfer_files({'path': 'C://Users//dinar//Desktop//50healthy_test/AA_20120815_SC.mat_SC_log_SC_22min_csf_0.0328.mat',
                'sid': 'sub-01'}, 'C://Users//dinar//Desktop//50healthy_test')

match: ['AA_20120815']
root: C://Users//dinar//Desktop//50healthy_test
sid: sub-01
f: AA_20120815_distances.txt
f: AA_20120815_SC.mat_SC_log_SC_22min_csf_0.0328.mat
f: AA_20120815_weights.txt
f: AY_20111004_distances.txt
f: AY_20111004_SC_new.mat_SC_log_SC_22min_csf_0.0325.mat
f: AY_20111004_weights.txt
f: sub-01


In [5]:
import shutil
import os

path = 'C:\\Users\\dinar\\Desktop\\50healthy_test\\1'

if not os.path.exists(path):
    os.mkdir(path)
    
shutil.move('C://Users//dinar//Desktop//50healthy_test/AA_20120815_distances.txt', path)

'C:\\Users\\dinar\\Desktop\\50healthy_test\\1\\AA_20120815_distances.txt'

In [8]:
from scipy.io import loadmat

loadmat('C:/Users/dinar/Desktop/50healthy_test_all/AA_20120815_SC.mat_SC_log_SC_22min_csf_0.0328.mat')

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Feb 14 04:52:19 2017',
 '__version__': '1.0',
 '__globals__': [],
 'Bold_time': array([[5.0000e+02, 1.0000e+03, 1.5000e+03, ..., 1.3190e+06, 1.3195e+06,
         1.3200e+06]]),
 'Bold_data': array([[[[ -141.48949614],
          [  -62.71886885],
          [  -82.33999015],
          ...,
          [  167.00314701],
          [   19.36960773],
          [   -9.46151917]]],
 
 
        [[[ -143.08279113],
          [  -64.29409459],
          [  -83.94336549],
          ...,
          [  165.27042134],
          [   17.67803488],
          [  -11.21377617]]],
 
 
        [[[ -153.07308041],
          [  -74.0562326 ],
          [  -94.2439947 ],
          ...,
          [  154.36469672],
          [    7.11115482],
          [  -22.63011437]]],
 
 
        ...,
 
 
        [[[-1464.41279874],
          [-1354.23103094],
          [-1532.3469634 ],
          ...,
          [-1294.22347323],
          [-1373.99152578],
 

In [4]:
path = '../output'
sys.path.append(path)

sim2bids.app.app.SoftwareVersion = 2.6
sim2bids.app.app.SoftwareRepository = 'https://github.com/the-virtual-brain/tvb-root/releases/tag/2.6'
sim2bids.app.app.SoftwareName = 'TVB'

app = MainArea()
pn.serve(app.view())

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:60869


INFO:tornado.access:200 GET / (127.0.0.1) 176.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/jsoneditor.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/datatabulator/tabulator-tables@4.9.3/dist/css/tabulator_simple.min.css (127.0.0.1) 3.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/npm/notyf@3/notyf.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/alerts.css (127.0.0.1) 2.99ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/ajax/libs/font-awesome/5.15.1/css/all.min.css (127.0.0.1) 7.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/card.css (127.0.0.1) 8.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/dataframe.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/debugger.css (127.0.0.1) 3.00ms
INFO:tornado.access:200 GET /static/extension

TIMES: ['bold_times.txt', 'subs_times_22min.txt', 'subs_times.txt', 'bold_times_22min.txt']
OrderedDict([('sub-01', {'distances': {'name': 'distances', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_times': {'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50hea

In [8]:
from collections import OrderedDict

sub = OrderedDict({'sub-01': {'distances': {'name': 'distances', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_times': {'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_ts': {'name': 'alpha-speed0.1-G10-bold_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_times': {'name': 'alpha-speed0.1-G10-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_ts': {'name': 'alpha-speed0.1-G10-subs_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_times_22min': {'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_ts_22min': {'name': 'alpha-speed100-G0.0398-bold_ts_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_times_22min': {'name': 'alpha-speed100-G0.0398-subs_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_ts_22min': {'name': 'alpha-speed100-G0.0398-subs_ts_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-bold_times': {'name': 'delta-speed20-G0.1-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_times': {'name': 'delta-speed20-G0.1-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_ts': {'name': 'delta-speed20-G0.1-subs_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_ts.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-bold_times': {'name': 'delta-speed20-G0.2-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_times': {'name': 'delta-speed20-G0.2-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_ts': {'name': 'delta-speed20-G0.2-subs_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_ts.txt', 'ext': 'txt'}, 'centres': {'name': 'centres', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\centres.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\centres.txt', 'ext': 'txt'}}})

In [9]:
sub

OrderedDict([('sub-01',
              {'distances': {'name': 'distances',
                'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
                'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt',
                'ext': 'txt'},
               'emp_fc': {'name': 'emp_fc',
                'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
                'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt',
                'ext': 'txt'},
               'weights': {'name': 'weights',
                'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
   

In [13]:
for k, v in sub['sub-01'].items():
    if 'times' in k:
        print(v, end='\n\n')

{'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'ext': 'txt'}

{'name': 'alpha-speed0.1-G10-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'ext': 'txt'}

{'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398

INFO:bokeh.server.views.ws:WebSocket connection closed: code=1001, reason=None


In [16]:
from sim2bids.convert import convert

convert.get_specific('alpha')

['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt']

In [9]:
from sim2bids.app import app



In [19]:
get_times('delta', 'bold_times')

['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt']

In [24]:
import pandas as pd

a1 = pd.read_csv('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt',
                header=None, sep='\t')
a2 = pd.read_csv('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt',
                header=None, sep='\t')

In [28]:
a1.equals(a2)

True

In [14]:
import re 

def get_times(rhythm, ftype='bold_times'):
    content = []
    
    for file in app.ALL_FILES:
        if rhythm in file and ftype in file:
            content.append(file)
    
    return content


def check_times(subs):
    # 1. get rhythm type
    rhythm = re.findall(r'alpha|delta|gamma|theta|beta', subs['path'])[0]
    
    # 2. get 'times' type
    times_type = [t for t in app.TIMES if t in subs['path']]
    
    print(times_type)
    
    if isinstance(times_type, list) and len(times_type) > 0:
        # 3. get all files of specific type 
        files = get_times(rhythm, times_type[0])
        
        print(files)
        
        # 4. if there are multiple versions, 
#         if len(files) == 1:
            
    

In [15]:
check_times({'name': 'delta-speed20-G0.2-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'ext': 'txt'})




[]


In [41]:
app.TIMES

['bold_times.txt',
 'subs_times_22min.txt',
 'subs_times.txt',
 'bold_times_22min.txt']

In [52]:
check_times({'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'ext': 'txt'})

['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt']


In [55]:
from scipy.io import loadmat

loadmat('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\AY_20111004_SC_new.mat_SC_log_SC_22min_csf_0.0325.mat')




{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Feb 14 04:48:46 2017',
 '__version__': '1.0',
 '__globals__': [],
 'Bold_time': array([[5.0000e+02, 1.0000e+03, 1.5000e+03, ..., 1.3190e+06, 1.3195e+06,
         1.3200e+06]]),
 'Bold_data': array([[[[   31.30501829],
          [   22.94625814],
          [ -110.86465291],
          ...,
          [   23.94189774],
          [   57.64111017],
          [  -74.77319948]]],
 
 
        [[[   29.59338953],
          [   21.29085872],
          [ -112.49132285],
          ...,
          [   22.44538672],
          [   55.95230906],
          [  -76.48548574]]],
 
 
        [[[   18.65865173],
          [   10.86446101],
          [ -122.97435788],
          ...,
          [   12.98478864],
          [   45.34490782],
          [  -87.68838787]]],
 
 
        ...,
 
 
        [[[-1470.05512465],
          [-1400.31684914],
          [-1562.75033049],
          ...,
          [-1218.97335231],
          [-1303.16777743],
 

In [60]:
from sim2bids.generate import subjects as subs

subs.find_matches(['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt'])

[]

In [62]:
re.findall(r'[A-Za-z]{2,3}_[0-9]{2,}', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt')





['AA_20120815']

In [10]:
split = ['bold', 'times', '22min']

[split[-idx] for idx in list(range(3, 0, -1))]

['bold', 'times', '22min']

In [6]:
from scipy.io import loadmat

mat = loadmat('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\YE_20120910_SC.mat_csf_0.0398_speed_100_22min.mat')






In [14]:
one = mat['subs_data'][:, 0, :].shape

In [15]:
two = mat['subs_data'][:, 1, :].shape

In [16]:
one == two

True

In [10]:
mat['Bold_data'][:, 0, :, 0].shape

(600, 68)

In [4]:
import re

re.findall(r'[0-9]+min', 'YE_20120910_SC.mat_csf_0.0398_speed_100_22min')

['22min']

INFO:bokeh.server.views.ws:WebSocket connection closed: code=1001, reason=None


In [4]:
rhythms = pn.Column('#### Specify rhythms', pn.widgets.CheckBoxGroup(
    value=[], options=['Alpha', 'Beta', 'Delta', 'Gamma', 'Theta'],
    inline=True, margin=(-10,0,0,10)))

speed = pn.Column('#### Specify `conduction speed` folder name', pn.widgets.TextInput(value='cspeed', margin=(-10,0,0,0)))

global_coupling = pn.Column('#### Specify `global coupling scaling factor` folder name', 
                            pn.widgets.TextInput(value='csf', margin=(-10,0,0,0)))

col = pn.Column('### Specify simulation information', rhythms, speed, global_coupling)

In [5]:
pn.serve(col)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:50198


INFO:tornado.access:200 GET / (127.0.0.1) 123.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/jsoneditor.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/datatabulator/tabulator-tables@4.9.3/dist/css/tabulator_simple.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/npm/notyf@3/notyf.min.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/ajax/libs/font-awesome/5.15.1/css/all.min.css (127.0.0.1) 6.01ms
INFO:tornado.access:200 GET /static/extensions/panel/css/alerts.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/card.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/debugger.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/dataframe.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extension